In [1]:
from torch import nn
import torchvision
import torch.nn.functional as F
from typing import Tuple, List
import numpy as np
import torch


In [2]:
model = torchvision.models.resnet34(pretrained=True)
img = torch.zeros((1, 3, 128, 1024))




In [3]:
output_channels=[128, 256, 128, 128, 64, 64]
feature_extractor = nn.Sequential(#sol
    nn.Conv2d(3, 32, kernel_size=3, padding=1),#sol
    nn.BatchNorm2d(32),#sol
    nn.LeakyReLU(0.2),#sol
    nn.Conv2d(32, 64, kernel_size=3, padding=1),#sol
    nn.BatchNorm2d(64),#sol
    nn.LeakyReLU(0.2),#sol
    #sol
    nn.MaxPool2d(2,2), # 64x512 out#sol
    nn.Conv2d(64, 128, kernel_size=3, padding=1),#sol
    nn.BatchNorm2d(128),#sol
    nn.LeakyReLU(0.2),#sol
    nn.Conv2d(128, 128, kernel_size=3, padding=1),#sol
    nn.BatchNorm2d(128),#sol
    nn.LeakyReLU(0.2),#sol
    nn.Conv2d(128, 256, kernel_size=3, padding=1),#sol
    nn.BatchNorm2d(256),#sol
    nn.LeakyReLU(0.2),#sol
    nn.Conv2d(256, 512, kernel_size=3, padding=1),#sol
    nn.BatchNorm2d(512),#sol
    nn.LeakyReLU(0.2),#sol
    #Ouput 32#
    nn.Conv2d(512, output_channels[0], kernel_size=3, padding=1,stride=2),#so    
) #sol
        
additional_layers = nn.ModuleList([#sol
    nn.Sequential( # 16 x 128 out#sol
        nn.BatchNorm2d(output_channels[0]),#sol
        nn.LeakyReLU(0.2),#sol
        nn.Conv2d(output_channels[0], 512, kernel_size=3, padding=1),#sol
        nn.BatchNorm2d(512),#sol
        nn.LeakyReLU(0.2),#sol
        nn.Conv2d(512, output_channels[1], kernel_size=3, padding=1, stride=2),#sol
    ),#sol
    nn.Sequential( # 8x64 out#sol
        nn.BatchNorm2d(output_channels[1]),#sol
        nn.LeakyReLU(0.2),#sol
        nn.Conv2d(output_channels[1], 256, kernel_size=3, padding=1),#sol
        nn.BatchNorm2d(256),#sol
        nn.LeakyReLU(0.2),#sol
        nn.Conv2d(256, output_channels[2], kernel_size=3, padding=1, stride=2),#sol
    ),#sol
    nn.Sequential( # 4 x 32 out#sol
        nn.BatchNorm2d(output_channels[2]),#sol
        nn.LeakyReLU(0.2),#sol
        nn.Conv2d(output_channels[2], 128, kernel_size=3, padding=1),#sol
        nn.BatchNorm2d(128),#sol
        nn.LeakyReLU(0.2),#sol
        nn.Conv2d(128, output_channels[3], kernel_size=3, padding=1, stride=2),#sol
    ),#sol
    nn.Sequential( # 2 x 16 out#sol
        nn.BatchNorm2d(output_channels[3]),#sol
        nn.LeakyReLU(0.2),#sol
        nn.Conv2d(output_channels[3], 128, kernel_size=3, padding=1),#sol
        nn.BatchNorm2d(128),#sol
        nn.LeakyReLU(0.2),#sol
        nn.Conv2d(128, output_channels[4], kernel_size=3, stride=2, padding=1),#sol
    ),#sol
    nn.Sequential( # 1 x 8 out#sol
        nn.BatchNorm2d(output_channels[4]),#sol
        nn.LeakyReLU(0.2),#sol
        nn.Conv2d(output_channels[4], 128, kernel_size=2, padding=1),#sol
        nn.BatchNorm2d(128),#sol
        nn.LeakyReLU(0.2),#sol
        nn.Conv2d(128, output_channels[5], kernel_size=2,stride=2),#sol
    ),#sol

])#sol

In [4]:
img = torch.zeros((1, 3, 128, 1024))

print(img.shape)
out_features = []
x = feature_extractor(img)#sol
out_features.append(x)#sol

print(out_features[0].shape)

torch.Size([1, 3, 128, 1024])
torch.Size([1, 128, 32, 256])


In [5]:
for additional_layer in additional_layers.children():#sol
    x = additional_layer(x)#sol
    out_features.append(x)#sol

In [6]:
for i in out_features:
    print(i.shape)

torch.Size([1, 128, 32, 256])
torch.Size([1, 256, 16, 128])
torch.Size([1, 128, 8, 64])
torch.Size([1, 128, 4, 32])
torch.Size([1, 64, 2, 16])
torch.Size([1, 64, 1, 8])


In [7]:
from collections import OrderedDict
x = OrderedDict()
x['feat0'] = out_features[0]
x['feat1'] = out_features[1]
x['feat2'] = out_features[2]
x['feat3'] = out_features[3]
x['feat4'] = out_features[4]
x['feat5'] = out_features[5]

In [8]:
m=torchvision.ops.FeaturePyramidNetwork([128,256,128,128,64,64],256)

In [9]:
m=torchvision.ops.FeaturePyramidNetwork([10,20,30],5)

In [10]:
out= m(x)

RuntimeError: Given groups=1, weight of size [5, 30, 1, 1], expected input[1, 64, 1, 8] to have 30 channels, but got 64 channels instead

In [ ]:
print([(k, v.shape) for k, v in out.items()])

In [ ]:
out['feat0']=(nn.Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))(out['feat0']))
out['feat1']=(nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))(out['feat1']))
out['feat2']=(nn.Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))(out['feat2']))
out['feat3']=(nn.Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))(out['feat3']))
out['feat4']=(nn.Conv2d(256, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))(out['feat4']))
out['feat5']=(nn.Conv2d(256, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))(out['feat5']))






In [ ]:
print([(k, v.shape) for k, v in out.items()])

In [33]:

from torch import nn
import torchvision
import torch.nn.functional as F
from typing import Tuple, List
import numpy as np
import torch

In [62]:
x = torch.zeros((1, 3, 128, 1024))
backbone = torchvision.models.resnet34(pretrained=True)
base_layer0 = nn.Sequential( backbone.conv1,  backbone.bn1,  backbone.relu,  backbone.maxpool)
base_layer1 = nn.Sequential( backbone.layer1,backbone.maxpool)
base_layer2 = nn.Sequential( backbone.layer2)
base_layer3 = nn.Sequential( backbone.layer3)
base_layer4 = nn.Sequential( backbone.layer4)
print(backbone.conv1)

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)


In [55]:
c1 = base_layer0(x)
c2 = base_layer1(c1)
c3 = base_layer2(c2)
c4 = base_layer3(c3)
c5 = base_layer4(c4)
aux= nn.Conv2d(512, 1024, kernel_size=1, stride=2)
c6= aux(c5)
c7= nn.Conv2d(1024,1024,kernel_size=1,stride=2)(c6)
aux= 
from collections import OrderedDict
x= OrderedDict()
x['feat0'] = c2
x['feat1'] = c3
x['feat2'] = c4
x['feat3'] = c5
x['feat4'] = c6
x['feat5'] = c7

m=torchvision.ops.FeaturePyramidNetwork([64,128,256,512,1024],256)
out_features = []
       

torch.Size([1, 1024, 1, 8])


In [61]:
print(c6.shape)

torch.Size([1, 1024, 1, 8])


In [ ]:
out=m(x)

In [ ]:
print([(k, v.shape) for k, v in out.items()])

In [ ]:
out_features=[]
for k, v in out.items():
    out_features.append(v)
